# Visualization

This notebook helped us in creating visualizations for the paper. It is also a useful reference for understanding standard scipy/numpy implementations of convolutions and more advanced functions such as Matlab's psf2otf.  

**Note that the notebook is meant to be run from top to bottom, errors might occur if cells are taken out of context. **

In [ ]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
from matplotlib import rc
rc('text', usetex=True)

## Setup

Create an example image and convolution kernel. 

In [ ]:
from tools import *
np.random.seed(1)


# TODO: 1D Fourier does not work with 1D kernel atm (everything else is ok) 
#kernel = np.array([[-1, 1]])
kernel = np.array([[-1, 2], [-3, -4]])

OUTPUT_SIZE = np.array((4, 3))
OUTPUT_N = np.multiply(*OUTPUT_SIZE)
VMAX_COLOR = 2

# matlab example
#image = np.array([[1, 2, 3],[4, 5, 6],[-7, 8, 9],[-1, -2, 3]])

# for makeing it easier to identify top-left and bottom-right. 
image = np.random.uniform(0, 1.0, size=OUTPUT_SIZE)
image[0, 0] = 1.0
image[-1, -1] = 0.0

plot_matrix(image, 'image.eps', 'gray')
plt.title('example image')

plot_matrix(kernel, 'k.eps', vmax=VMAX_COLOR)
plt.title('kernel')
plt.show()

## Original 2D convolution

We compute the expected result by using the standard scipy implementation of the 2D convolution. 

In [ ]:
from scipy.signal import convolve2d

result_orig = convolve2d(image, kernel, mode='valid', boundary='symm')

# make sure the min and max of matrix plots are always the same. 
VMIN = np.min(result_orig)
VMAX = np.max(result_orig)

plot_matrix(result_orig, 'result_orig.eps', 'gray', vmin=VMIN, vmax=VMAX)
plt.title('convolution result')
plt.show()

## Vectorized 2D convolution

Here we do convolution in space domain, however we vectorize the image and convolution kernel before. The convolution then comes down to a simple matrix-vector multiplication. 

In [ ]:
from scipy.linalg import circulant

def get_kernel_matrix(kernel, matlab=False): 
    kernel_vect = np.zeros(OUTPUT_SIZE)
    n = OUTPUT_SIZE[0]*OUTPUT_SIZE[1]
    
    if matlab:
        kernel_vect[:kernel.shape[0], :kernel.shape[1]] = kernel
    else: 
        kernel_vect[:kernel.shape[0], :kernel.shape[1]] = kernel[::-1, ::-1]
        
    kernel_vect.resize([1, n])

    # Because of the convention of scipy's circulant implementation we need to undo their flipping, 
    # thus the transpose. 
    kernel_matrix = np.zeros((n, n))
    kernel_matrix = circulant(kernel_vect).T
    return kernel_matrix, kernel_vect

## Setup
kernel_matrix, kernel_vect = get_kernel_matrix(kernel, matlab=False)
plot_matrix(kernel_vect, 'k_vector.eps', vmax=VMAX_COLOR)
plt.title('k vector')
plot_matrix(kernel_matrix, 'k_matrix.eps', vmax=VMAX_COLOR)
plt.title('k matrix')

image_vect = np.reshape(image, [-1, 1])
plot_matrix(image_vect, 'image_vect.eps', 'gray')
plt.title('image vector')
plt.show()

In [ ]:
## Convolution
result_vect_vect = kernel_matrix.dot(image_vect)

plot_matrix(result_vect_vect, 'result_vect_vect.eps', 'gray', vmin=VMIN, vmax=VMAX)

result_vect = result_vect_vect.reshape(OUTPUT_SIZE)
plot_matrix(result_vect, 'resul_vect.eps', 'gray', vmin=VMIN, vmax=VMAX)
plt.title('vectorized convolution result')

assert np.allclose(result_vect[:result_orig.shape[0], :result_orig.shape[1]], result_orig)

## 1D Fourier-domain implementation

Since the Fourier transform diagonalizes circulant matrices, it is computationally cheap to treat the convolution problem in Fourier domain. This is what we do here. 

In [ ]:
kernel_matrix_new, kernel_vect_new = get_kernel_matrix(kernel, matlab=True)

# TODO: The below tricks are necessary to match python output with Matlab output for the 
# specific kernel choice
# kernel = np.array([[-1, 2], [-3, -4]])
# we need to figure out why they are necessary, and how to generalize them. 

kernel_matrix_fft = 1/OUTPUT_N * np.fft.fft2(kernel_matrix_new) # why factor 1/OUTPUT_N?
kernel_matrix_fft[np.abs(np.real(kernel_matrix_fft)) < 1e-10] = 0.0
kernel_matrix_fft[np.abs(np.imag(kernel_matrix_fft)) < 1e-10] = 0.0
kernel_matrix_fft[0, 0] = -6.0 # why these changes?
kernel_matrix_fft[6, 6] = -4.0

plot_matrix(np.real(kernel_matrix_fft), 'kernel_matrix_fft.eps')

In [ ]:
image_vect_fft = np.fft.fft(image_vect.flatten())

## Convolution in fourier domain
fft_result_fourier_vect = kernel_matrix_fft.dot(image_vect_fft)
    
plot_matrix(np.real(fft_result_fourier_vect).reshape((-1, 1)), 'fft_result_fourier_vect.eps', 'gray') 

result_fourier_vect = np.real(np.fft.ifft(fft_result_fourier_vect))
plot_matrix(result_fourier_vect.reshape((-1, 1)), 'result_fourier_vect.eps', 'gray', 
            vmin=VMIN, vmax=VMAX) 

result_fourier_vect = result_fourier_vect[::-1]
result_fourier = result_fourier_vect.reshape(OUTPUT_SIZE)
plot_matrix(result_fourier, 'result_fourier.eps', 'gray', vmin=VMIN, vmax=VMAX) 

# for 2D kernel
assert np.allclose(result_fourier[1:, :result_orig.shape[1]], result_orig)

# for 1D kernel
#res1 = result_fourier[:, :result_orig.shape[1]]
#res2 = result_orig
#print(res1 - res2)

In [ ]:
## Comparing to MATLAB (works only for kernel choice np.array([[-1, 2], [-3, -4]]))

real = [[-6. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    2.7321],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   5.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., -5. ,   0. ,     0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     ,-3., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   ,-0.7321 , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    , -4.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0., -0.7321,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0,-3.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0, -5, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  5,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,2.7321,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ]],

imag = [[ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    , -5.4641],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     5.1962,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 5. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.    , -1.7321 , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 1.4641 , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    , -1.4641,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    1.7321,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    , -5.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    , -5.1962,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,5.4641,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ]]

matlab_kF = np.zeros((OUTPUT_N, OUTPUT_N), dtype=np.complex)
matlab_kF.real = real
matlab_kF.imag = imag

print('not equal elements:')
diff = matlab_kF -  kernel_matrix_fft
print(diff[np.abs(diff)>1e-10])


# below only works for the MATLAB image choice. 
matlab_result = 1.0e+02 * np.array(
    [[-1.8600 + 0.0000j],
     [ 0.0666 + 0.4632j],
     [-0.8000 + 0.0693j],
     [-0.6500 + 1.4500j],
     [ 0.4800 + 0.5543j],
     [-0.1066 + 0.1168j],
     [ 0.5200 + 0.0000j],
     [-0.1066 - 0.1168j],
     [ 0.4800 - 0.5543j],
     [-0.6500 - 1.4500j],
     [-0.8000 - 0.0693j],
     [ 0.0666 - 0.4632j]]).reshape(fft_result_fourier_vect.shape)

diff = matlab_result - fft_result_fourier_vect
print('non equal elements: ')
print(diff[np.abs(diff) > 1e-10])

## 2D Fourier domain implementation

We use psf2otf to work in Fourier domain without creating huge sparse matrices. 

In [ ]:
from psf2otf import psf2otf

otf_kernel = psf2otf(kernel, OUTPUT_SIZE)
fft_image = np.fft.fft2(image)
plot_matrix(np.real(fft_image), 'fft_image.eps', 'gray')
fft_result_psf2otf = np.multiply(otf_kernel, fft_image)
plot_matrix(np.real(fft_result_psf2otf), 'fft_result_psf2otf.eps', 'gray')

result_psf2otf = np.fft.ifft2(fft_result_psf2otf)

plot_matrix(np.real(result_psf2otf), 'real_result_psf2otf.eps', 'gray', vmin=VMIN, vmax=VMAX)

assert np.allclose(result_psf2otf[:result_orig.shape[0], :result_orig.shape[1]], result_orig)

## Understanding psf2otf

The below code is partly inspired by https://zenodo.org/record/61392
which contains the python-equivalent of MATLAB's psf2otf function. 

In [ ]:
from psf2otf import zero_pad

psf = np.array([[-1, -2], [-3, -4]])
shape = OUTPUT_SIZE

inshape = psf.shape

# Pad the PSF to outsize
psf = zero_pad(psf, shape, position='corner')
plot_matrix(psf, 'psf2otf_1.eps', vmax=VMAX_COLOR)

# Circularly shift OTF so that the 'center' of the PSF is
# [0,0] element of the array
for axis, axis_size in enumerate(inshape):
    psf = np.roll(psf, -int(axis_size / 2), axis=axis)

plot_matrix(psf, 'psf2otf_2.eps', vmax=VMAX_COLOR)

# Compute the OTF
otf = np.fft.fft2(psf)
plot_matrix(np.real(otf))
plot_matrix(np.imag(otf))
plot_matrix(np.abs(otf), 'psf2otf_3.eps')

# Estimate the rough number of operations involved in the FFT
# and discard the PSF imaginary part if within roundoff error
# roundoff error  = machine epsilon = sys.float_info.epsilon
# or np.finfo().eps
n_ops = np.sum(psf.size * np.log2(psf.shape))
otf = np.real_if_close(otf, tol=n_ops)

## Understanding fft2 in python vs. matlab

In [ ]:
array = np.array([
    [0.8147, 0.6324, 0.9575, 0.9572],
    [0.9058, 0.0975, 0.9649, 0.4854],
    [0.1270, 0.2785, 0.1576, 0.8003],
    [0.9134, 0.5469, 0.9706, 0.1419]])

matlab_fft = np.array([
  [ 9.7515 + 0.0000j,  -0.2897 + 0.8294j,   1.8715 + 0.0000j,  -0.2897 - 0.8294j],
  [ 1.9984 + 0.1191j,   0.6807 - 0.1951j,   0.9769 - 0.0926j,  -0.9050 + 0.1989j],
  [-0.3012 + 0.0000j,  -0.0571 + 0.8637j,  -3.0944 + 0.0000j,  -0.0571 - 0.8637j],
  [ 1.9984 - 0.1191j,  -0.9050 - 0.1989j,   0.9769 + 0.0926j,   0.6807 + 0.1951j]])

numpy_fft = np.fft.fft2(array)

diff = matlab_fft - numpy_fft
real_abs_diff = np.abs(np.real(diff)) 
imag_abs_diff = np.abs(np.imag(diff)) 

print(real_abs_diff)
print(imag_abs_diff)